In [1]:
#import split data
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.stats import truncnorm
import pandas as pd

In [2]:
import requests
import pandas as pd


# Specify the countries and indicator
countries = "CAN;FIN;ITA;KEN;NOR;SGP"  # ISO codes for Canada, Finland, Italy, Kenya, Norway, Singapore
indicator = "NY.GDP.PCAP.CD"  # GDP per capita (current US$)
years = ["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022","2023"]

gdp_data = {}

for year in years:
        
    url = f"https://api.worldbank.org/v2/country/{countries}/indicator/{indicator}?format=json&date={year}"
    # Fetch the data
    response = requests.get(url)
    data = response.json()

    # Extract relevant data
    for entry in data[1]:
        country = entry['country']['value']
        gdp_per_capita = entry['value']
        year = entry['date']
        if country not in gdp_data:
            gdp_data[country] = {}
            gdp_data[country][year] = gdp_per_capita
        else:
            if year not in gdp_data[country]:
                gdp_data[country][year] = gdp_per_capita


print(gdp_data)


{'Canada': {'2010': 47560.6666009406, '2011': 52223.8588398531, '2012': 52670.3447335415, '2013': 52638.1187235237, '2014': 50960.8431174661, '2015': 43594.1941045394, '2016': 42314.0615817218, '2017': 45129.628116623, '2018': 46539.1761570405, '2019': 46352.8693445211, '2020': 43537.839298904, '2021': 52496.8441693242, '2022': 55509.393176404, '2023': 53431.1857063879}, 'Finland': {'2010': 46506.2919016566, '2011': 51060.3242589767, '2012': 47551.6740841369, '2013': 49691.0145200739, '2014': 50073.7760815871, '2015': 42560.3456767103, '2016': 43451.2562442158, '2017': 46085.0174739036, '2018': 49654.2497035329, '2019': 48358.1807773701, '2020': 48828.6846862799, '2021': 53099.1351400148, '2022': 50438.4753952355, '2023': 52925.6897638424}, 'Italy': {'2010': 36184.7118698678, '2011': 38851.3881339353, '2012': 35235.7989058242, '2013': 35747.707952689, '2014': 35750.7197500382, '2015': 30387.1293187854, '2016': 31126.3246947273, '2017': 32554.14668453, '2018': 34746.3441392416, '2019': 

In [3]:

from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import pandas as pd

def compute_gdp_per_capita(X,gdp_data):
    def get_gdp(row):
        country = str(row["country"])
        year = str(int(row["year"])+2010)
        return gdp_data[country][year]
    X["gdp_per_capita"] = X.apply(get_gdp, axis=1)
    return X

def generate_features(X, train=False):
    X = X.copy()
        
    X["date"] = pd.to_datetime(X["date"], format="%Y-%m-%d")
    X["dayofweek"] = X["date"].dt.dayofweek
    X["month"] = X["date"].dt.month - 1
    X["year"] = X["date"].dt.year - 2010
    X["is_weekend"] = X["dayofweek"].isin([5,6])
    X["sin_dayofweek"] = np.sin(X["dayofweek"] * (2 * np.pi / 7))
    X["cos_dayofweek"] = np.cos(X["dayofweek"] * (2 * np.pi / 7))
    X["sin_month"] = np.sin(X["month"] * (2 * np.pi / 12))
    X["cos_month"] = np.cos(X["month"] * (2 * np.pi / 12))
    X["sin_year"] = np.sin(X["year"] * (2 * np.pi / 10))
    X["cos_year"] = np.cos(X["year"] * (2 * np.pi / 10))
    
    if train:
        #Nan values in num_sold replace by mean of num_sold for each country, store and product
        #X["num_sold"] = X.groupby(["country", "store", "product"])["num_sold"].transform(lambda x: x.fillna(x.mean()))
        X = X.drop(columns=["id"])
        #X = X.dropna(subset=["num_sold"])
        
    X = compute_gdp_per_capita(X,gdp_data)
    
    X = X.drop(columns=["date","month","year","dayofweek"])
        
    return X

FOLDER = "playground-series-s5e1/"
data = pd.read_csv(FOLDER + "train.csv")
train_data = pd.read_csv(FOLDER + "train.csv")
test_data = pd.read_csv(FOLDER + "test.csv")

print(train_data.isna().sum())

print(train_data["country"].unique())
train_data = generate_features(train_data,train=True)
test_data = generate_features(test_data)

print("Train data")
print(train_data.head())
print("Test data")
print(test_data.head())

id             0
date           0
country        0
store          0
product        0
num_sold    8871
dtype: int64
['Canada' 'Finland' 'Italy' 'Kenya' 'Norway' 'Singapore']
Train data
  country              store             product  num_sold  is_weekend  \
0  Canada  Discount Stickers   Holographic Goose       NaN       False   
1  Canada  Discount Stickers              Kaggle     973.0       False   
2  Canada  Discount Stickers        Kaggle Tiers     906.0       False   
3  Canada  Discount Stickers            Kerneler     423.0       False   
4  Canada  Discount Stickers  Kerneler Dark Mode     491.0       False   

   sin_dayofweek  cos_dayofweek  sin_month  cos_month  sin_year  cos_year  \
0      -0.433884      -0.900969        0.0        1.0       0.0       1.0   
1      -0.433884      -0.900969        0.0        1.0       0.0       1.0   
2      -0.433884      -0.900969        0.0        1.0       0.0       1.0   
3      -0.433884      -0.900969        0.0        1.0       0.0

In [4]:
train_data.isna().sum()

country              0
store                0
product              0
num_sold          8871
is_weekend           0
sin_dayofweek        0
cos_dayofweek        0
sin_month            0
cos_month            0
sin_year             0
cos_year             0
gdp_per_capita       0
dtype: int64

In [11]:
data.isna().sum()
    

id          0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

In [71]:
nan_values = data["num_sold"].isna()
missing_values = data.loc[nan_values]

#can I plot a pie chart of the missing values for each country, store, product
missing_values.groupby(["country", "store", "product"])["id"].count()


country  store                 product           
Canada   Discount Stickers     Holographic Goose     2557
                               Kerneler                 1
         Premium Sticker Mart  Holographic Goose      380
         Stickers for Less     Holographic Goose     1308
Kenya    Discount Stickers     Holographic Goose     2557
                               Kerneler                63
                               Kerneler Dark Mode       1
         Premium Sticker Mart  Holographic Goose      646
         Stickers for Less     Holographic Goose     1358
Name: id, dtype: int64

In [52]:
from xgboost import XGBRegressor
import optuna
from sklearn.metrics import mean_squared_error
X = train_data.drop(columns=['num_sold'])
y = train_data['num_sold']

cat_cols = ['country', 'store', 'product']
num_cols = ["sin_dayofweek", "cos_dayofweek", "sin_month", "cos_month", "sin_year", "cos_year","gdp_per_capita"]

# Create different preprocessors for different models
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# For models that need one-hot encoding (XGBoost, RandomForest)
categorical_transformer_ohe = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Create different preprocessors
preprocessor_ohe = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer_ohe, cat_cols)
])

# For LightGBM and CatBoost, we'll just scale numerics and pass categoricals as is
preprocessor_native = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols)
])

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_ohe),
    ('regressor', XGBRegressor(
        n_estimators=5000,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ))
])

def objective(trial):
    
    #params max_depth, min_child_weight, gamma, learning_rate
    max_depth = trial.suggest_int("max_depth", 3, 10)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 10)
    gamma = trial.suggest_int("gamma", 0, 10)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
    n_estimators = trial.suggest_int("n_estimators", 100, 5000)
    subsample = trial.suggest_float("subsample", 0.5, 1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1)
    colsample_bylevel = trial.suggest_float("colsample_bylevel", 0.5, 1)
    colsample_bynode = trial.suggest_float("colsample_bynode", 0.5, 1)
    lambda_l1 = trial.suggest_float("lambda_l1", 0, 10)
    lambda_l2 = trial.suggest_float("lambda_l2", 0, 10)
    
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor_ohe),
        ('regressor', XGBRegressor(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_child_weight=min_child_weight,
            gamma=gamma,
            random_state=42,
            objective='reg:squarederror',
            booster="gbtree",
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            colsample_bylevel=colsample_bylevel,
            colsample_bynode=colsample_bynode,
            reg_alpha=lambda_l1,
            reg_lambda=lambda_l2
        ))
    ])
    #Time-series cross-validation
    time_series = TimeSeriesSplit(n_splits=5)
    fold_scores_val= []
    fold_scores_train= []
    loss_train= []
    loss_val= []
    model_name = "xgboost"
    for fold_index, (train_index, val_index) in enumerate(time_series.split(X)):
        X_train_fold = X.iloc[train_index]
        y_train_fold = y.iloc[train_index]
        X_val_fold = X.iloc[val_index]
        y_val_fold = y.iloc[val_index]
        
        y_train_fold_transformed = np.log1p(y_train_fold)
        y_val_fold_transformed = np.log1p(y_val_fold)
        
        model_pipeline.fit(X_train_fold, y_train_fold_transformed)
        
        y_pred_val = model_pipeline.predict(X_val_fold)    
        y_pred_train = model_pipeline.predict(X_train_fold)
        
        y_pred_val_transformed = np.expm1(y_pred_val)
        y_pred_train_transformed = np.expm1(y_pred_train)
        
        mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val_transformed)
        mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train_transformed)
        
        mse_train = mean_squared_error(y_train_fold, y_pred_train_transformed)
        mse_val = mean_squared_error(y_val_fold, y_pred_val_transformed)
        
        fold_scores_val.append(mape_val)
        fold_scores_train.append(mape_train)
        loss_train.append(mse_train)
        loss_val.append(mse_val)
        
        # print(f"Fold {fold_index + 1} loss train: {mse_train:.4f}")
        # print(f"Fold {fold_index + 1} loss val: {mse_val:.4f}")
        # print("--------------------------------")
        # print(f"Fold {fold_index + 1} metric val: {mape_val:.4f}")
        # print(f"Fold {fold_index + 1} metric train: {mape_train:.4f}")
        # print("--------------------------------")
        
    print(f"Average MAPE for {model_name}: {np.mean(fold_scores_val):.4f}")
    print(f"Average MAPE for {model_name} train: {np.mean(fold_scores_train):.4f}")
    # print(f"Average MSE for {model_name} train: {np.mean(loss_train):.4f}")
    # print(f"Average MSE for {model_name} val: {np.mean(loss_val):.4f}")
    
    return np.mean(fold_scores_val)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40)
print(f"Best parameters: {study.best_params}")
print(f"Best value: {study.best_value:.4f}")





[I 2025-01-16 16:34:52,482] A new study created in memory with name: no-name-72847431-4a7e-4fb7-9978-d05694d5cd81
[I 2025-01-16 16:35:10,655] Trial 0 finished with value: 0.11807659092352815 and parameters: {'max_depth': 6, 'min_child_weight': 6, 'gamma': 2, 'learning_rate': 0.03210282192067468, 'n_estimators': 3261, 'subsample': 0.949424738025167, 'colsample_bytree': 0.5029583642161533, 'colsample_bylevel': 0.5382799441251553, 'colsample_bynode': 0.8986506537411352, 'lambda_l1': 2.1855846278041904, 'lambda_l2': 4.125673362137149}. Best is trial 0 with value: 0.11807659092352815.


Average MAPE for xgboost: 0.1181
Average MAPE for xgboost train: 0.0797


[I 2025-01-16 16:35:19,992] Trial 1 finished with value: 0.13838218584123013 and parameters: {'max_depth': 6, 'min_child_weight': 1, 'gamma': 9, 'learning_rate': 0.24911483575403864, 'n_estimators': 1701, 'subsample': 0.6497220519552172, 'colsample_bytree': 0.6022149668472376, 'colsample_bylevel': 0.8213186240503774, 'colsample_bynode': 0.7156281719468283, 'lambda_l1': 6.067743097258278, 'lambda_l2': 1.0415614401389683}. Best is trial 0 with value: 0.11807659092352815.


Average MAPE for xgboost: 0.1384
Average MAPE for xgboost train: 0.1001


[I 2025-01-16 16:35:24,823] Trial 2 finished with value: 0.12732270208841187 and parameters: {'max_depth': 9, 'min_child_weight': 8, 'gamma': 8, 'learning_rate': 0.21064401442874833, 'n_estimators': 695, 'subsample': 0.7634215097262399, 'colsample_bytree': 0.6736460183024124, 'colsample_bylevel': 0.6779695598593198, 'colsample_bynode': 0.8379940412781617, 'lambda_l1': 4.824720648424304, 'lambda_l2': 3.7571000116842876}. Best is trial 0 with value: 0.11807659092352815.


...

Average MAPE for xgboost: 0.1028
Average MAPE for xgboost train: 0.0514


[I 2025-01-16 16:54:49,366] Trial 38 finished with value: 0.09904048200134594 and parameters: {'max_depth': 6, 'min_child_weight': 2, 'gamma': 0, 'learning_rate': 0.2438853390687511, 'n_estimators': 2710, 'subsample': 0.7743750640122247, 'colsample_bytree': 0.917406591244253, 'colsample_bylevel': 0.5210269372262178, 'colsample_bynode': 0.7470861674929081, 'lambda_l1': 4.1022383032551675, 'lambda_l2': 6.541399171171751}. Best is trial 38 with value: 0.09904048200134594.


Average MAPE for xgboost: 0.0990
Average MAPE for xgboost train: 0.0514


[I 2025-01-16 16:55:21,904] Trial 39 finished with value: 0.09707012984999255 and parameters: {'max_depth': 6, 'min_child_weight': 2, 'gamma': 0, 'learning_rate': 0.22526539269160925, 'n_estimators': 2000, 'subsample': 0.7504843677097007, 'colsample_bytree': 0.9058833944527335, 'colsample_bylevel': 0.5023703000686878, 'colsample_bynode': 0.7460010757653904, 'lambda_l1': 4.13721855361009, 'lambda_l2': 6.767325827239311}. Best is trial 39 with value: 0.09707012984999255.


Average MAPE for xgboost: 0.0971
Average MAPE for xgboost train: 0.0515
Best parameters: {'max_depth': 6, 'min_child_weight': 2, 'gamma': 0, 'learning_rate': 0.22526539269160925, 'n_estimators': 2000, 'subsample': 0.7504843677097007, 'colsample_bytree': 0.9058833944527335, 'colsample_bylevel': 0.5023703000686878, 'colsample_bynode': 0.7460010757653904, 'lambda_l1': 4.13721855361009, 'lambda_l2': 6.767325827239311}
Best value: 0.0971
